In [1]:
Rooms = Vector{Vector{Char}}
Hallway = Vector{Char}

assignments = Vector{Char}(['A', 'B', 'C', 'D'])

function display_state(rooms::Rooms, hallway::Hallway)
    str = ("#"^13 * '\n' *
           '#' * String(hallway[1:2]) * '.' * hallway[3] * '.' * 
           hallway[4] * '.' * hallway[5] * '.' * String(hallway[6:7]) *"#\n" )
    
    for i = 1:length(rooms[1])
        new = "#$(rooms[1][i])#$(rooms[2][i])#$(rooms[3][i])#$(rooms[4][i])#"
        if i == 1
            str *= '#'^2 * new * '#'^2 * '\n'
        else
            str *= ' '^2 * new * '\n'
        end
    end
    str *= "  " * '#'^9
    println(str)
end

function hallway_between_rh(ir::Int64, ih::Int64, hallway::Hallway)
    i, j = ir+1.5, ih
    if j > i
        return hallway[Int(ceil(i)):j-1]
    else
        return hallway[j+1:Int(floor(i))]
    end
end

function hallway_between_rr(ir1::Int64, ir2::Int64, hallway::Hallway)::Vector{Char}
    ir1, ir2 = sort([ir1, ir2])
    return hallway[ir1+2:ir2+1]
end

function valid_r2h(rooms::Rooms, hallway::Hallway)::Vector{String}
    valids = Vector{String}([])
    if all(hallway .!= '.')
        return valids
    end
    for (ir, r) in enumerate(rooms)
        start = "r$(ir)"
        if all(r .== '.') || all((r .== assignments[ir]) .| (r .== '.'))
            continue
        end
        for ih in 1:7
            if hallway[ih] != '.'
                continue
            end
            between = hallway_between_rh(ir, ih, hallway)
            if (length(between) == 0) || all(between .== '.')
                push!(valids, start * "h$(ih)")
            end
        end
    end
    return valids
end

function valid_r2r(rooms::Rooms, hallway::Hallway)::Vector{String}
    valids = Vector{String}([])
    for (ir, r) in enumerate(rooms)
        if all(r .== '.')
            continue
        end
        ind = findfirst(x->x!='.', r)
        val = r[ind]
        final = findfirst(x->x==val, assignments)
        between = hallway_between_rr(ir, final, hallway)
        if (ir != final) && all(between .== '.')  && all(@. (rooms[final] == val) | (rooms[final] == '.'))
            push!(valids, "r$(ir)r$(final)")
        end
    end
    return valids
end

function valid_h2r(rooms::Rooms, hallway::Hallway)::Vector{String}
    valids = Vector{String}([])
    for (ih, h) in enumerate(hallway)
        if h == '.'
            continue
        end
        room = findfirst(x->x==h, assignments)
        room_contents = rooms[room]
        if !all(@. (room_contents == h) | (room_contents == '.'))
            continue
        end
        blockers = hallway_between_rh(room, ih, hallway)
        if (length(blockers) == 0) || all(blockers .== '.')
            push!(valids, "h$(ih)r$(room)")
        end
    end
    return valids
end

type_costs = Dict(['A'=>1,'B'=>10,'C'=>100,'D'=>1000])

hallway_offset_rh = Vector{Vector{Int64}}([
    [3,2,2,4,6,8,9], # room 1
    [5,4,2,2,4,6,7], # room 2
    [7,6,4,2,2,4,5], # room 3
    [9,8,6,4,2,2,3], # room 4
])

function move_cost(move::String, rooms::Rooms, hallway::Hallway)
    code = move[1] * move[3]
    i1, i2 = parse(Int, move[2]), parse(Int, move[4])
    if code == "rr"
        cost = count(x->x=='.', rooms[i1]) + count(x->x=='.', rooms[i2])
        cost += (2*abs(i1-i2)+1)
        cost *= type_costs[rooms[i1][findfirst(x->x!='.', rooms[i1])]]
    elseif code == "rh"
        cost = count(x->x=='.', rooms[i1])
        cost += hallway_offset_rh[i1][i2]
        cost *= type_costs[rooms[i1][findfirst(x->x!='.', rooms[i1])]]
    elseif code == "hr"
        cost = count(x->x=='.', rooms[i2])
        cost += hallway_offset_rh[i2][i1] - 1
        cost *= type_costs[hallway[i1]]
    end
    return cost
end

function valid_moves(rooms::Rooms, hallway::Hallway)::Vector{Tuple{Int64,String}}
    moves = vcat(valid_r2h(rooms, hallway), valid_r2r(rooms, hallway), valid_h2r(rooms, hallway))
    return sort([(move_cost(m, rooms, hallway),m) for m in moves])
end

function perform_move(move::String, rooms::Rooms, hallway::Hallway)::Tuple{Rooms,Hallway}
    r = deepcopy(rooms)
    h = deepcopy(hallway)
    i1 = parse(Int, move[2])
    i2 = parse(Int, move[4])
    if move[1] == 'r'
        ind = findfirst(x->x!='.', r[i1])
        val = r[i1][ind]
        r[i1][ind] = '.'
    else
        val = h[i1]
        h[i1] = '.'
    end
    if move[3] == 'r'
        if all(r[i2] .== '.')
            r[i2][end] = val
        else
            ind = findfirst(x->x!='.', r[i2]) - 1
            if ind == 0
                display_state(rooms, hallway)
                println(move)
            end
            r[i2][ind] = val
        end
    else
        h[i2] = val
    end
    return r, h
end

function hash_state(rooms::Rooms, hallway::Hallway)
    out = String(hallway)
    for r in rooms
        out *= String(r)
    end
    return out
end

function unhash_state(hstate::String)::Tuple{Rooms,Hallway}
    hallway = Hallway(collect(hstate[1:7]))
    rsize = (length(hstate) - 7) ÷ 4
    rooms = Rooms([])
    for ir = 0:3
        room_contents = collect(hstate[8+ir*rsize:8+(ir+1)*rsize-1])
        push!(rooms, room_contents)
    end
    return rooms, hallway
end

unhash_state (generic function with 1 method)

In [2]:
using DataStructures

function solve(start::String)::Int64
    room_size = length(start) ÷ 4
    start  = '.'^7 * start
    destination = '.'^7 * 'A'^room_size * 'B' ^ room_size * 'C'^room_size * 'D'^room_size
    
    distances = DefaultDict{String,Int64}(typemax(Int64))
    distances[start] = 0

    visited = Dict{String,Nothing}([])
    frontier = PriorityQueue{String,Int64}([start=>0])

    while length(frontier) > 0
        current = dequeue!(frontier)
        if current == destination
            return distances[current]
        end
        rooms, hallway = unhash_state(current)
        for (c, m) in valid_moves(rooms, hallway)   
            ndist = distances[current] + c
            r, c = perform_move(m, rooms, hallway)
            hn = hash_state(r, c)
            if (ndist < distances[hn]) || !haskey(visited, hn)
                visited[hn] = nothing
                distances[hn] = ndist
                if !haskey(frontier, hn)
                    enqueue!(frontier, hn=>ndist)
                end
            end
        end
    end
    return -1
end

solve (generic function with 1 method)

In [3]:
println("Part 1 test: $(solve("BACDBCDA"))")
println("Part 1 solution: $(solve("BBACADDC"))")

Part 1 test: 12521
Part 1 solution: 11417


In [4]:
println("Part 2 test: $(solve("BDDACCBDBBACDACA"))")
println("Part 2 solution: $(solve("BDDBACBCABADDACC"))")

Part 2 test: 44169
Part 2 solution: 49529
